## Exchange Energy 

    VII. Calculate the exchange potential and energy from the previously calculated density.
    
    VIII. Write the calculation of exchange as a function.

The exchange correlation functional is a correction to the electronic energy that approximates the effect of electron interactions.

The exchange functional in the local density approximation(LDA) is given by, 

$$E_X^{LDA}[n] = -\frac34 {\left(\frac{3}{\pi}\right)}^\frac13 \int n^{\frac43} dx$$

In the derivation of the Kohn-Sham equations, the potential corresponding to each energy term is defined as the derivative of the energy with respect to the density. The exchange potential is therefore given by the derivative of the exchange energy with respect to the density:

$$v_x^{LDA}(x) = \frac{\partial E_X^{LDA}[n]}{\partial n(x)} = -{\left(\frac{3}{\pi}\right)}^\frac13 n^{\frac13}(x)$$

In [1]:
from scipy.integrate import quad
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.integrate import simps

In [2]:
def D(x, N=100):
    '''
    Returns the operator form of first-derivative

            Parameters:
                    x (array): grid of 1D array
                    N (int): number of points on the grid

            Returns:
                    D (array): first-derivative operator
    '''
    h = x[1] - x[0]
    k = [np.ones(N-1), -np.ones(N-1)]
    offset = [1, -1]
    D = diags(k, offset).toarray()
    D = D/(2 * h)
    # # Boundary values where it is not well defined
    D[0, 0] = 0
    D[0, 1] = 0
    D[1, 0] = 0
    D[N-1, N-2] = 0
    D[N-2, N-1] = 0
    D[N-1, N-1] = 0
    return D


def D2(x, N=100):
    '''
    Returns the operator form of second-derivative

            Parameters:
                    x (array): grid of 1D array
                    N (int): number of points on the grid

            Returns:
                    D (array): first-derivative operator
    '''
    h = x[1] - x[0]
    k1 = [np.ones(N-1), -2*np.ones(N), np.ones(N-1)]
    offset = [-1, 0, 1]
    D2 = diags(k1, offset).toarray()
    D2 = D2/(h ** 2)
    # Boundary values where it is not well defined
    D2[0, 0] = 0
    D2[0, 1] = 0
    D2[1, 0] = 0
    D2[N-1, N-2] = 0
    D2[N-2, N-1] = 0
    D2[N-1, N-1] = 0
    return D2


def normalise_psi(psi, x):
    '''
    Normalises the given wavefunction

            Parameters:
                    psi (array): wavefunction psi
                    x (array): 1D grid of array

            Returns:
                    normalised psi
    '''
    int_psi_square = simps(abs(psi) ** 2, x)
    return psi/np.sqrt(int_psi_square)


def get_occupation_num(nElectron, maxElectron=2):
    '''
    Returns a list of occupation numbers for a given number of electrons

            Parameters:
                    nElectron (int): number of electrons
                    maxElectron (int): max number of allowed elecrons in one state

            Returns:
                    fn (array): occupation number
    '''
    nFloor = np.floor_divide(nElectron, 2)
    fn = maxElectron * np.ones(nFloor)
    if nElectron % 2:
        fn = np.append(fn, 1)
    return fn


def get_density(nElectron, psi, x):
    '''
    Returns electron density for a given number of electrona and wavefunction

            Parameters:
                    nElectron (int): number of electrons
                    psi (array): wavefunction
                    x (array): 1D grid of array

            Returns:
                    eDensity (array): electron density
    '''
    psiNorm = normalise_psi(psi, x)  # Normalisation
    fn = get_occupation_num(nElectron)
    eDensity = np.zeros(N)
    for f_n, psi in zip(fn, psiNorm.T):
        eDensity += f_n * (psi ** 2)
    return eDensity


def integrate(x, y):
    '''
    Returns the integration by simpson's method

            Parameters:
                    x (array): 1D grid of arra
                    y (array): integrand

            Returns:
                    result (float): result of the integration
    '''
    result = simps(y, x)
    return result


def calculate_exchange(eDensity, x):
    '''
    Returns exchange energy and potential for a given electron density

            Parameters:
                    eDensity (array): electron density
                    x (array): 1D grid of array

            Returns:
                    energy (float): exchange energy
                    potential (float): exchange potential
    '''
    energy = -3/4 * (3/np.pi) ** (1/3) * integrate(x, eDensity ** (4/3))
    potential = -(3/np.pi) ** (1/3) * eDensity ** (1/3)
    return energy, potential

In [3]:
L = 5
N = 200
x = np.linspace(-L, L, N)
X = np.diagflat(x*x)
H = -D2(x, N)/2 + X
E, V = np.linalg.eigh(H)
psi = V
nElectron = 6

In [4]:
density = get_density(nElectron, psi, x)

In [5]:
Ex, Vx = calculate_exchange(density, x)

In [6]:
Ex

-5.107000404110526

In [7]:
Vx

array([-0.00000000e+00, -8.70402151e-05, -1.43012654e-04, -1.97979851e-04,
       -2.57948886e-04, -3.26843519e-04, -4.08090740e-04, -5.05133416e-04,
       -6.21689501e-04, -7.61926612e-04, -9.30604331e-04, -1.13320517e-03,
       -1.37606420e-03, -1.66650285e-03, -2.01296988e-03, -2.42519145e-03,
       -2.91433132e-03, -3.49316136e-03, -4.17624232e-03, -4.98011394e-03,
       -5.92349336e-03, -7.02747986e-03, -8.31576355e-03, -9.81483518e-03,
       -1.15541932e-02, -1.35665436e-02, -1.58879880e-02, -1.85581935e-02,
       -2.16205377e-02, -2.51222219e-02, -2.91143446e-02, -3.36519259e-02,
       -3.87938738e-02, -4.46028837e-02, -5.11452597e-02, -5.84906487e-02,
       -6.67116778e-02, -7.58834841e-02, -8.60831305e-02, -9.73888980e-02,
       -1.09879450e-01, -1.23632862e-01, -1.38725519e-01, -1.55230875e-01,
       -1.73218085e-01, -1.92750510e-01, -2.13884111e-01, -2.36665743e-01,
       -2.61131375e-01, -2.87304244e-01, -3.15192995e-01, -3.44789817e-01,
       -3.76068630e-01, -